<a href="https://colab.research.google.com/github/jmoro0408/sloped_pipe_volume/blob/main/Sloped_Pipe_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook aims to show Jupyter's interactive plotting capabilites, and mimics the pipe part full calculation provided in Excel

## Imports

In [1]:
#@title
import numpy as np
import scipy as sp
from ipywidgets import HBox, Label, widgets, Button, VBox
from IPython.display import Markdown as md

# Background and Purpose

This tool calculates the liquid volume in a partially full horizontal or sloped pipe or tank. 

It uses the pipe invert elevation, diameter, and water level at the pipe lower level to estimate the volume iof liquid within the pipe. 

In [2]:
#@title Please choose desired units.  { run: "auto" }
units = "Metric" #@param ["Metric", "Imperial"]


In [49]:
#@title
# Define Input boxes

pipe_invert_elev=widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1e6,
    step=0.1,
    disabled=False, 
)
pipe_diameter=widgets.BoundedFloatText(
    value=2,
    min=0,
    max=1e6,
    step=0.1,
    disabled=False, 
)
pipe_length= widgets.BoundedFloatText(
    value=200,
    min=0,
    max=1e6,
    step=0.1,
    disabled=False, 
)
liquid_density=widgets.BoundedFloatText(
    value=998,
    min=0,
    max=1e6,
    step=0.1,
    disabled=False, 
)
lower_liquid_level= widgets.BoundedFloatText(
    value=1,
    min=0,
    max=1e6,
    step=0.1,
    disabled=False, 
)

grade_percent = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=5, #grade % slider
    step=0.01,
    description='Grade %:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

Please input the required variables below:

In [40]:
#@title
inputs_boxes= [
               HBox([Label('Lower End Pipe Invert Elevation (mGeodetic):'), pipe_invert_elev]), 
               HBox([Label('Pipe diameter (m):'), pipe_diameter]), 
               HBox([grade_percent]), 
               HBox([Label('Pipe length (m):'), pipe_length]), 
               HBox([Label('Liquid Density (kg/m3):'), liquid_density]),
               HBox([Label('Liquid at pipe lower elevation (m):'), lower_liquid_level]),

]


VBox(inputs_boxes)


In [41]:
pipe_radius = pipe_diameter.value / 2

def calculate_tilt_angle(grade_percent = grade_percent):
  tilt_angle = np.arctan(grade_percent.value / 100)
  tilt_angle = np.degrees(tilt_angle)
  return tilt_angle

tilt_angle = calculate_tilt_angle()

def calculate_pipe_upper_invert(pipe_invert_elev = pipe_invert_elev, tilt_angle = tilt_angle, pipe_length = pipe_length):
  pipe_upper_invert = np.sin(np.radians(tilt_angle) * pipe_length.value) + pipe_invert_elev.value
  return pipe_upper_invert

pipe_upper_invert = calculate_pipe_upper_invert()

def calculate_upper_liquid_level(lower_liquid_level = lower_liquid_level, tilt_angle = tilt_angle, pipe_length = pipe_length):
  upper_liquid_level = lower_liquid_level.value - (pipe_length.value *np.tan(np.radians(tilt_angle)))
  return upper_liquid_level

upper_liquid_level = calculate_upper_liquid_level()

def calculate_sloped_pipe_volume(pipe_radius=pipe_radius, upper_liquid_level = upper_liquid_level, tilt_angle=tilt_angle, pipe_length = pipe_length.value):
  tilt_angle=np.radians(tilt_angle)
  _k = 1 - (upper_liquid_level/pipe_radius)
  _c = _k - ((pipe_length * np.tan(tilt_angle)/pipe_radius))
  integral0 = pipe_radius**3 / np.tan(tilt_angle)
  integral1 = _k* np.arccos(_k)
  integral2 = (1/3) * (np.sqrt(1 - _k**2)) * (_k**2 +2)
  integral3 = _c * np.arccos(_c)
  integral4 = (1/3) * (np.sqrt(1 - _c**2)) * (_c**2 +2)
  volume = integral0 * (integral1 - integral2 - integral3 + integral4)
  return volume

sloped_pipe_liquid_volume = calculate_sloped_pipe_volume()

# Horizontal pipe characteristics

In [42]:
pipe_cross_sectional_area = np.pi / 4 *  (pipe_diameter.value**2)
total_volume = pipe_cross_sectional_area * pipe_length.value
liquid_volume = 0.5 * pipe_radius
arc_angle = np.degrees(2* np.arccos(pipe_radius - lower_liquid_level.value) /  pipe_radius)
arc_length = pipe_radius * np.radians(arc_angle)
chord_length = 2*np.sqrt(lower_liquid_level.value*(2*pipe_radius-lower_liquid_level.value)) 
pipe_liquid_volume=(0.5*(pipe_radius*arc_length - chord_length*(pipe_radius-lower_liquid_level.value)))*pipe_length.value
pipe_air_volume=total_volume-pipe_liquid_volume
liquid_fill_percent = (pipe_liquid_volume/total_volume)*100
liquid_mass_in_pipe= pipe_liquid_volume*liquid_density.value
liquid_cross_section_area = 0.5*(pipe_radius*arc_length-chord_length*(pipe_radius-lower_liquid_level.value))
air_cross_section_area = pipe_cross_sectional_area - liquid_cross_section_area

# Sloped Pipe Characteristics

## Definitions

The volume inside the pipe is defined by the integral:

$V = \frac{R^2}{2} \theta(x) - \int_0^L \! sin\theta(x) \, \mathrm{d}x.$

where 
$\theta(x)=2\arccos{\frac{R-h}{R}}=2\arccos{\frac{R-(x\tan{\alpha}+h_{0})}{R}}$

Substituting in we get:



In [43]:
#@title
sloped_pipe_liquid_volume = calculate_sloped_pipe_volume()
sloped_pipe_air_volume = total_volume - sloped_pipe_liquid_volume
sloped_pipe_percent_full = sloped_pipe_liquid_volume / total_volume * 100
sloped_pipe_liquid_mass = sloped_pipe_liquid_volume * liquid_density.value

In [44]:
print(f"The pipe contains {round(sloped_pipe_liquid_volume, 2)} m3 liquid and is {round(sloped_pipe_percent_full, 2)}% full. \n The mass of liquid inside the pipe is {round(sloped_pipe_liquid_mass,2)}kg ")

The pipe contains 235.24 m3 liquid and is 37.44% full. 
 The mass of liquid inside the pipe is 234773.29kg 


## Interactive

In [52]:
import pandas as pd
import plotly.graph_objects as go

In [46]:
def build_graph_df(no_points = 100,pipe_length = pipe_length.value, upper_liquid_level = upper_liquid_level,lower_liquid_level = lower_liquid_level.value, pipe_invert_elev = pipe_invert_elev.value, pipe_upper_invert=pipe_upper_invert,pipe_diameter=pipe_diameter.value ):
  graph_df = pd.DataFrame(
      data = {
          "Pipe Chainage":np.linspace(0, pipe_length, no_points), 
          "Liquid Level":np.linspace(lower_liquid_level, upper_liquid_level, no_points), 
          "Pipe Invert": np.linspace(pipe_invert_elev, pipe_upper_invert, no_points), 
          "Pipe Crown":np.linspace((pipe_invert_elev + pipe_diameter) , (pipe_upper_invert +pipe_diameter), no_points),
          
      }
  )
  graph_df = graph_df.round(3)
  return graph_df

In [47]:
graph_df=build_graph_df()

In [48]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=graph_df["Pipe Chainage"], y=graph_df["Pipe Invert"],name = "Pipe Invert",showlegend=False, line=dict(color='gray', width=4)))
fig.add_trace(go.Scatter(x=graph_df["Pipe Chainage"], y=graph_df["Pipe Crown"] ,name = "Pipe Crown",showlegend=False, line=dict(color='gray', width=4)))
fig.add_trace(go.Scatter(x=graph_df["Pipe Chainage"], y=graph_df["Liquid Level"], name = "Liquid Level",line=dict(color='royalblue', width=4)))


fig.update_xaxes(showspikes=True, spikecolor="green")
fig.update_yaxes(showspikes=True,spikecolor="green")


fig.update_layout(
    template = "simple_white", 
    title="Liquid Level Along Pipe",
    xaxis_title="Pipe Chainage (m)",
    yaxis_title="Elevation (mGeodetic)",
    
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))


sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 50},
    steps = steps
)]

# Old Stuff

In [14]:
grade_percent_slider = widgets.FloatSlider(
#     value=grade_percent.value,
#     min=0,
#     max=5, #grade % slider
#     step=0.01,
#     description='Grade %:',
#     disabled=False,
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='.1f',
# )
# f = go.FigureWidget(fig)

SyntaxError: ignored

In [ ]:
# def response(change):
#   graph_df=build_graph_df()
#   y1 = graph_df["Liquid Level"].values
#   y2 = graph_df["Pipe Invert"].values
#   y3 = graph_df["Pipe Crown"].values
#   with f.batch_update():
#     f.data[0].y = y2
#     f.data[1].y=y3
#     f.data[2] = y1
#   f.show()


In [ ]:
# widgets.VBox([grade_percent_slider])